In [5]:
# from __init__ import * 
from snmcseq_utils import create_logger
from snmcseq_utils import get_cluster_mc_c 
# from CEMBA_update_mysql import connect_sql
# from CEMBA_init_ensemble_v2 import pull_genebody_info
# from CEMBA_init_ensemble_v2 import pull_binc_info

In [6]:
log = create_logger()

In [ ]:
ens = 'Ens1'
context = 'CH'
df = get_cluster_mc_c(ens, context, genome_regions='genebody', cluster_type='mCHmCG_lv_npc50_k30', database='CEMBA')

02/28/2018 05:04:55 PM Pulling genebody information (1202 cells)...
02/28/2018 05:04:55 PM /cndd/Public_Datasets/CEMBA/snmCSeq/Ensembles/Ens1/gene_level created!
02/28/2018 05:04:55 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_4_CEMBA171206_3C_5_A11_AD002_indexed (1/1202)
02/28/2018 05:04:55 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_G4_AD008_indexed (2/1202)
02/28/2018 05:04:55 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_C3_AD002_indexed (3/1202)
02/28/2018 05:04:55 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_1_CEMBA171206_3C_3_H3_AD002_indexed (4/1202)
02/28/2018 05:04:55 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_4_CEMBA171206_3C_5_B7_AD002_indexed (5/1202)
02/28/2018 05:04:56 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171206_3C_4_CEMBA171206_3C_5_E8_AD001_indexed (6/1202)
02/28/2018 05:04:56 PM Loaded cell: 171213_CEMBA_mm_P56_P63_3C_MOp_CEMBA171